# Rigorous Showcase: NumDynInflation Utilities

This notebook demonstrates the capabilities of the `NumDynInflation` repository. To maintain maximum rigor and avoid code duplication, we **import the physics engine directly from the repository** rather than re-defining it here.

## 1. Setup and Libary Architecture

We import high-level solvers, model definitions, and professional plotting utilities. The `inflation_models` module allows us to easily switch between different potentials.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from inflation_models import QuadraticModel, HiggsModel
from inf_dyn_background import run_background_simulation, get_derived_quantities
from inf_dyn_MS_full import run_ms_simulation, get_ms_derived_quantities
from inf_dyn_plot import set_style, plot_background_n_vs_t, plot_eps_h, plot_power_spectrum, plot_phase_space

# Activate professional styling
set_style()

## 2. Model Visualization: The Quadratic Potential

We instantiate the `QuadraticModel` and visualize its potential $f(x) = x^2$.

In [ ]:
model_quad = QuadraticModel()

x_vals = np.linspace(-20, 20, 100)
plt.figure(figsize=(8, 5))
plt.plot(x_vals, model_quad.f(x_vals), 'k-', lw=2)
plt.title(f'Model: {model_quad.name}')
plt.xlabel(r'$x = \phi/m_p$')
plt.ylabel(r'$f(x)$')
plt.grid(True)
plt.show()


## 3. Background Dynamics

We execute the numerical integration of the Friedmann The `run_background_simulation` function now accepts a model object, ensuring that the correct derivatives and parameters are used.

In [ ]:
T_span = np.linspace(0, 1000, 100000)
sol_bg = run_background_simulation(model_quad, T_span)
bg_results = get_derived_quantities(sol_bg, model_quad)

# Visualize expansion
plot_background_n_vs_t(T_span, bg_results['N'])

## 4. Derived Physics: Slow-Roll Parameters

Calculated quantities like $\epsilon_H$ (which tells us when inflation ends) and $\eta_H$ are extracted from the raw simulation data.

In [ ]:
plot_eps_h(bg_results['N'], bg_results['epsH'], bg_results['etaH'])

## 5. Phase Space Analysis

The $(\phi, \dot{\phi})$ phase space is a critical diagnostic. We use the library's plotting utility to visualize the attractor trajectory.

In [ ]:
plot_phase_space(sol_bg[0], sol_bg[1] * model_quad.S)

## 6. Model Comparison: Higgs Inflation

We now switch to the **Higgs Inflation** model by simply instantiating `HiggsModel`. The simulation code remains exactly the same, demonstrating the power of our modular architecture.

In [ ]:
model_higgs = HiggsModel()

# 1. Plot the Potential Comparison
x_vals = np.linspace(0, 15, 100)
plt.figure(figsize=(8, 5))
plt.plot(x_vals, model_higgs.f(x_vals)/np.max(model_higgs.f(x_vals)), 'r-', lw=2, label='Higgs Potential (Normalized)')
plt.plot(x_vals, (x_vals**2)/np.max(x_vals**2), 'k--', lw=2, label='Quadratic Potential (Normalized)')
plt.xlabel('Field Value $x$')
plt.title('Potential Shape Comparison')
plt.legend()
plt.grid(True)
plt.show()

# 2. Run Higgs Simulation (using same function!)
# Higgs inflation usually takes longer in dimensionless time T
T_span_h = np.linspace(0, 5000, 100000)
sol_higgs = run_background_simulation(model_higgs, T_span_h)
higgs_results = get_derived_quantities(sol_higgs, model_higgs)

# 3. Compare Observables (Tensor-to-Scalar Ratio)
plt.figure(figsize=(10, 6))
plt.plot(bg_results['N'], bg_results['r'], 'k--', lw=3, label='Quadratic: $r \approx 0.16$')
plt.plot(higgs_results['N'], higgs_results['r'], 'r-', lw=3, label='Higgs: $r \approx 0.003$')
plt.yscale('log')
plt.xlabel('N (e-folds)')
plt.ylabel('Tensor-to-Scalar Ratio $r$')
plt.title('Observational Check: \n Higgs Inflation fits Planck data ($r < 0.036$), Quadratic does not.')
plt.legend()
plt.grid(True)
plt.show()